In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path=r'C:\Users\Hemanth Nair\Desktop\Deep Learning files\Sequence to Sequence\fra-eng\fra.txt'

In [2]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines= f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text, target_text, _=line.split('\t')
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
#input_texts, 
#target_texts
len(target_characters)
#len(input_characters)

92

In [4]:
input_characters= sorted(list(input_characters))
target_characters= sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length= max([len(txt) for txt in input_texts])
max_decoder_seq_length= max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length) 

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


Assigning index for input and output tokens

In [6]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i, char in enumerate(target_characters)])

In [7]:
input_token_index #, target_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

One hot representtion

In [8]:
encoder_input_data= np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]]=1.
    encoder_input_data[i, t+1:, input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]]=1.
        if t>0:
            decoder_target_data[i, t-1, target_token_index[char]]=1.
    decoder_input_data[i, t+1:, target_token_index[' ']]=1.
    decoder_target_data[i, t:, target_token_index[' ']]=1.

In [10]:
encoder_input_data[0].shape

(16, 71)

Encoder Section

In [11]:
encoder_inputs= Input(shape=(None, num_encoder_tokens))
encoder= LSTM(latent_dim, return_state= True)
encoder_outputs, state_h, state_c= encoder(encoder_inputs)
encoder_states = [state_h, state_c]

Decoder Section

In [12]:
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm= LSTM(latent_dim, return_sequences= True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation= 'softmax')
decoder_outputs= decoder_dense(decoder_outputs)

Creating the model

In [13]:
model= Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs=epochs, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 21s 3ms/sample - loss: 1.1815 - accuracy: 0.7250 - val_loss: 1.0734 - val_accuracy: 0.7021
Epoch 2/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.8584 - accuracy: 0.7688 - val_loss: 0.8443 - val_accuracy: 0.7664
Epoch 3/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.6916 - accuracy: 0.8048 - val_loss: 0.7373 - val_accuracy: 0.7837
Epoch 4/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.5994 - accuracy: 0.8253 - val_loss: 0.6531 - val_accuracy: 0.8105
Epoch 5/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.5475 - accuracy: 0.8404 - val_loss: 0.6137 - val_accuracy: 0.8185
Epoch 6/100
8000/8000 [==============================] - 18s 2ms/sample - loss: 0.5104 - accuracy: 0.8502 - val_loss: 0.5767 - val_accuracy: 0.8290
Epoch 7/100
8000/8000 [==============================] - 19s 2ms

Epoch 56/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.1076 - accuracy: 0.9663 - val_loss: 0.5592 - val_accuracy: 0.8736
Epoch 57/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.1054 - accuracy: 0.9668 - val_loss: 0.5627 - val_accuracy: 0.8741
Epoch 58/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.1035 - accuracy: 0.9674 - val_loss: 0.5670 - val_accuracy: 0.8740
Epoch 59/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.1015 - accuracy: 0.9678 - val_loss: 0.5674 - val_accuracy: 0.8746
Epoch 60/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.0992 - accuracy: 0.9685 - val_loss: 0.5710 - val_accuracy: 0.8748
Epoch 61/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.0976 - accuracy: 0.9691 - val_loss: 0.5785 - val_accuracy: 0.8744
Epoch 62/100
8000/8000 [==============================] - 19s 2ms/sample - loss: 0.0951 - accuracy: 0.9696

Testing Section

In [32]:
encoder_model=Model(encoder_inputs, encoder_states)
decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c= decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states=[state_h, state_c]
decoder_outputs= decoder_dense(decoder_outputs)
decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)
reverse_input_char_index= dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index= dict((i,char) for char, i in target_token_index.items())
def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1, num_decoder_tokens))
    target_seq[0,0, target_token_index['\t']]=1.
    stop_condition= False
    decoded_sentence=''
    while not stop_condition:
        output_tokens, h,c =decoder_model.predict([target_seq]+ states_value)
        sampled_token_index= np.argmax(output_tokens[0,-1,:])
        sampled_char= reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char== '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition= True
            
        target_seq= np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] =1.
        
        states_value=[h,c]
    return decoded_sentence
for seq_index in range(100):
    input_seq= encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence= decode_sequence(input_seq)
    print('-')
    print('Input sentense:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentense: Go.
Decoded sentence: Va !

-
Input sentense: Hi.
Decoded sentence: Salut.

-
Input sentense: Hi.
Decoded sentence: Salut.

-
Input sentense: Run!
Decoded sentence: Cours !

-
Input sentense: Run!
Decoded sentence: Cours !

-
Input sentense: Who?
Decoded sentence: Qui ?

-
Input sentense: Wow!
Decoded sentence: Ça alors !

-
Input sentense: Fire!
Decoded sentence: Au feu !

-
Input sentense: Help!
Decoded sentence: À l'aide !

-
Input sentense: Jump.
Decoded sentence: Saute.

-
Input sentense: Stop!
Decoded sentence: Arrête-toi !

-
Input sentense: Stop!
Decoded sentence: Arrête-toi !

-
Input sentense: Stop!
Decoded sentence: Arrête-toi !

-
Input sentense: Wait!
Decoded sentence: Attends !

-
Input sentense: Wait!
Decoded sentence: Attends !

-
Input sentense: Go on.
Decoded sentence: Vas-y maintenant.

-
Input sentense: Go on.
Decoded sentence: Vas-y maintenant.

-
Input sentense: Go on.
Decoded sentence: Vas-y maintenant.

-
Input sentense: Hello!
Decoded sentence